### Batch Runner of ModularCirc

This file samples the input parameters of the Korakianitis model then batch solves this using ModularCirc. The raw output, pressure traces, cardiac output are all saved. Additionally a PCA is run on the pressure traces and are also saved.

In [1]:
from ModularCirc.Models.KorakianitisMixedModel import KorakianitisMixedModel, KorakianitisMixedModel_parameters, TEMPLATE_TIME_SETUP_DICT
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
from ModularCirc import BatchRunner

In [3]:
br = BatchRunner('Sobol', 0) 
#'LHS' : LatinHypercube,
#'Sobol' : Sobol,
#'Halton': Halton,

In [4]:
path = os.getcwd()
path

'/Users/pmzff/Documents/GitHub/ModularCirc/Emulation'

In [5]:
# Parameters_01 = Korakianitis Model
br.setup_sampler('parameters_pulmonary_sensitive.json')

FileNotFoundError: [Errno 2] No such file or directory: 'parameters_pulmonary_sensitive.json'

In [ ]:
# Set number of samples 
n_sample = 100
br.sample(n_sample)

In [ ]:
br._samples

In [ ]:
n_sample = 5000
n_params = 6
sample_name = 'waveform'

output_dir = f"../Results/{n_sample}_{n_params}params"

posterior_samples = pd.read_csv(f"{output_dir}/posterior_samples_{sample_name}.csv")

# remove any #s from column names
posterior_samples.columns = posterior_samples.columns.str.lstrip('#').str.strip()

for i, col in enumerate(br._samples.columns[:len(posterior_samples.columns)]):
    br._samples.loc[:, col] = posterior_samples.loc[:,col]

br.samples

In [ ]:
posterior_samples

In [ ]:
TEMPLATE_TIME_SETUP_DICT

In [ ]:
map_ = {
    'delay' : ['la.delay', 'ra.delay'],
    'td0'   : ['lv.td0',   'rv.td0' ],
    'tr'    : ['lv.tr',    'rv.tr'  ],
    'tpww'  : ['la.tpww',  'ra.tpww'],
}
br.map_sample_timings(
    ref_time=1.,
    map=map_
    )

In [ ]:
br._samples[['lv.td', 'rv.td']] = br._samples[['lv.tr', 'rv.tr']].values + br._samples[['lv.td0', 'rv.td0']].values
br._samples.drop(['lv.td0', 'rv.td0'], axis=1, inplace=True)

In [ ]:
br.samples

In [ ]:
# count number of sampled parameters
relevant_columns = []
for col in br.samples.columns:
    relevant_columns.append(col)
    if col == 'T': break

n_params = len(relevant_columns)

In [ ]:
br.map_vessel_volume()


In [ ]:
br.setup_model(model=KorakianitisMixedModel, po=KorakianitisMixedModel_parameters, time_setup=TEMPLATE_TIME_SETUP_DICT)

In [ ]:
input_header = ','.join(br.samples.columns)
input_header

In [ ]:
# Save sampled inputs to CSV
np.savetxt(f'{output_dir}/posterior_{sample_name}_full.csv', br.samples, header=input_header, delimiter=',')

In [ ]:
os.system(f'mkdir -p {output_dir}/Posterior_Simulations/posterior_sim_{sample_name}')
test = br.run_batch(n_jobs=5, output_path=f'{output_dir}/Posterior_Simulations/posterior_sim_{sample_name}')

In [ ]:
test

### Some of the simulations will not converge

In [ ]:
# Check for bool values in the list
bool_indices = [index for index, value in enumerate(test) if isinstance(value, bool)]

if bool_indices:
    print(f"Boolean values found at indices: {bool_indices}")
    print(f"Number of Booleans = {len(bool_indices)}")
else:
    print("No boolean values found in the list.")

In [ ]:
## Create directory for pessure traces 
os.system(f'mkdir -p {output_dir}/Posterior_Simulations/posterior_sim_{sample_name}/pressure_traces_pat')
os.system(f'mkdir -p {output_dir}/Posterior_Simulations/posterior_sim_{sample_name}/pressure_traces_rv')

### Save feasible pressure traces, CO and dt, dPAP, sPAP, mPAP

In [ ]:
# screen determines whetehr to screen for non-physiological pressure traces
screen = False

In [ ]:
# Create column headers
headers = list(range(100)) + ['CO', 'dt', 'EF', 'dPAP', 'sPAP', 'mPAP'] 

# List to collect all pressure traces
pressure_traces_list_pat = []
pressure_traces_list_rv = []

for ind in range(len(test)):
    if not isinstance(test[ind], bool):
     
     # PAT pressure
     p_pat_raw = test[ind].loc[ind]['p_pat'].values.copy()
     
     # RV pressure 
     p_rv_raw = test[ind].loc[ind]['p_rv'].values.copy()

     T = test[ind].loc[ind]['T'].values.copy()
     T_resample = np.linspace(T[0], T[-1], 100)

     # Interpolate pressure for 100 timesteps from 1000
     p_pat_resampled = np.interp(T_resample, T, p_pat_raw)
     p_rv_resampled = np.interp(T_resample, T, p_rv_raw)

     # Compute CO
     q_pat = test[ind].loc[ind]['q_pat'].values.copy()
     CO = np.sum(q_pat) * (T[1] - T[0]) / (T[-1] - T[0]) * 60. / 1000.  # L / min

     # Compute EF
     v_rv = test[ind].loc[ind]['v_rv'].values.copy()
     EF = (np.max(v_rv) - np.min(v_rv)) / np.max(v_rv)

     # Compute dPAP, sPAP, mPAP
     dPAP = min(p_rv_raw)
     sPAP = max(p_rv_raw)
     mPAP = np.mean(p_rv_raw)
    
     # Record time interval, approx T (input param) / 100, there are some rounding differences due to interpolation
     tl = T_resample - test[ind].loc[ind]['T'].iloc[0]
     dt = np.diff(tl)[0]

     
     # Only create array if conditions hold or screening is turned off
     if not screen or (2 < CO < 12 and 4 < dPAP < 67 and 9 < mPAP < 87 and 15 < sPAP < 140):
     
     # Create a 2D array for saving
        pressure_trace_pat = np.hstack((p_pat_resampled, [CO], [dt], [EF], [dPAP], [sPAP], [mPAP]))
        pressure_trace_rv = np.hstack((p_rv_resampled, [CO], [dt], [EF], [dPAP], [sPAP], [mPAP]))
        pressure_traces_list_pat.append(pressure_trace_pat)
        pressure_traces_list_rv.append(pressure_trace_rv)
        
        # Save individual pressure trace to CSV with headers
        individual_df_pat = pd.DataFrame([pressure_trace_pat], columns=headers)
        individual_df_pat.to_csv(f'{output_dir}/Posterior_Simulations/posterior_sim_{sample_name}/pressure_traces_pat/pressuretrace_{ind}.csv', index=False)
        individual_df_rv = pd.DataFrame([pressure_trace_rv], columns=headers)
        individual_df_rv.to_csv(f'{output_dir}/Posterior_Simulations/posterior_sim_{sample_name}/pressure_traces_rv/pressuretrace_{ind}.csv', index=False)

# Convert the list of pressure traces to a DataFrame
pressure_traces_df_pat = pd.DataFrame(pressure_traces_list_pat, columns=headers)
pressure_traces_df_rv = pd.DataFrame(pressure_traces_list_rv, columns=headers)

# Save the DataFrame to a single CSV file with headers
pressure_traces_df_pat.to_csv(f'{output_dir}/Posterior_Simulations/posterior_sim_{sample_name}/pressure_traces_pat/all_pressure_traces_pat.csv', index=False)
pressure_traces_df_rv.to_csv(f'{output_dir}/Posterior_Simulations/posterior_sim_{sample_name}/pressure_traces_rv/all_pressure_traces_rv.csv', index=False)